In [1]:
#importando bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
# from imblearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier, plot_importance
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
import matplotlib.cm as cm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')

In [3]:
#ler csv

#stats
stats_ger = pd.read_csv("rolling_avg/stats_avg_ger.csv")
stats_ita = pd.read_csv("rolling_avg/stats_avg_ita.csv")
stats_eng = pd.read_csv("rolling_avg/stats_avg_eng.csv")
stats_fra = pd.read_csv("rolling_avg/stats_avg_fra.csv")
stats_spa = pd.read_csv("rolling_avg/stats_avg_spa.csv")
stats_eur = pd.read_csv("rolling_avg/stats_avg_eur.csv")
stats_cup = pd.read_csv("rolling_avg/stats_avg_cup.csv")

#nets
nets_ger = pd.read_csv("rolling_avg/net_avg_ger.csv")
nets_ita = pd.read_csv("rolling_avg/net_avg_ita.csv")
nets_eng = pd.read_csv("rolling_avg/net_avg_eng.csv")
nets_fra = pd.read_csv("rolling_avg/net_avg_fra.csv")
nets_spa = pd.read_csv("rolling_avg/net_avg_spa.csv")
nets_eur = pd.read_csv("rolling_avg/net_avg_eur.csv")
nets_cup = pd.read_csv("rolling_avg/net_avg_cup.csv")